### 0. Importación de librerías 

En las siguientes líneas de código se importan las librerías y herramientas necesarias para desarrollar el caso de uso.

In [1]:
# Librerías para manejo de datos
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import numpy as np
np.random.seed(3301)
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.cluster import KMeans
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix,recall_score 
from sklearn.utils.class_weight import compute_class_weight
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
# Para la validación cruzada
from sklearn.model_selection import KFold 
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns   
from sklearn import tree, svm
from imblearn.over_sampling import SMOTE


from sklearn.metrics import silhouette_samples, silhouette_score


from sklearn.preprocessing import MinMaxScaler

from sklearn.cluster import AgglomerativeClustering
import numpy as np

import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering

from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt

from sklearn.cluster import DBSCAN

# Carga de los datos
A través de la librería **pandas** podemos realizar la carga de datos desde diferentes fuentes de información, en este caso se realizará la carga de un archivo plano csv, el separador del archivo es ; y se remplazaron los valores "-" por valores nulos.

## Dataset tweets

In [2]:
# Se cargan los datos. 
df_tweets=pd.read_csv('./archive/mbsa.csv', sep=',', encoding = 'utf-8', header=0)

In [3]:
# Cantidad de datos y número de variables
df_tweets.shape

(19344048, 3)

In [4]:
# Mostrar los datos
df_tweets.head()

,Date,text,Sentiment
0,2019-05-27,È appena uscito un nuovo video! LES CRYPTOMONN...,Positive
1,2019-05-27,Cardano: Digitize Currencies; EOS https://t.co...,Positive
2,2019-05-27,Another Test tweet that wasn't caught in the s...,Positive
3,2019-05-27,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,Positive
4,2019-05-27,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,Positive


In [5]:
# Podemos ver los tipos de todas la variables.
df_tweets.dtypes

Date         object
text         object
Sentiment    object
dtype: object

In [6]:
df_tweets["Sentiment"].value_counts()

Negative    9729297
Positive    8723207
Neutral         400
Name: Sentiment, dtype: int64

## Dataset Precios Bitcoin
dado que hay muy pocas muestras nuetrales, decidimos usar el dataset de precios de bitcoin para poder definir cuando hay un cambio positivo, cuando negativo y cuando neutral

In [7]:
# Se cargan los datos. 
df_precios=pd.read_csv('./archive2/coin_Bitcoin.csv', sep=',', encoding = 'utf-8', header=0)

In [9]:
# Cantidad de datos y número de variables
df_precios.shape

(2991, 10)

In [10]:
# Mostrar los datos
df_precios.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Bitcoin,BTC,2013-04-29 23:59:59,147.488007,134.000000,134.444000,144.539993,0.0,1.603769e+09
1,2,Bitcoin,BTC,2013-04-30 23:59:59,146.929993,134.050003,144.000000,139.000000,0.0,1.542813e+09
2,3,Bitcoin,BTC,2013-05-01 23:59:59,139.889999,107.720001,139.000000,116.989998,0.0,1.298955e+09
3,4,Bitcoin,BTC,2013-05-02 23:59:59,125.599998,92.281898,116.379997,105.209999,0.0,1.168517e+09
4,5,Bitcoin,BTC,2013-05-03 23:59:59,108.127998,79.099998,106.250000,97.750000,0.0,1.085995e+09


In [11]:
# Podemos ver los tipos de todas la variables.
df_precios.dtypes

SNo            int64
Name          object
Symbol        object
Date          object
High         float64
Low          float64
Open         float64
Close        float64
Volume       float64
Marketcap    float64
dtype: object

# Limpieza y preparación de los datos

Primero vamos a ejecutar los pasos de limpieza de los datos, relacionados el tratamiento de ausencias y registros duplicados.

## Tweets

In [12]:
# Es recomendable que todos los pasos de limpieza y preparación se realicen sobre otro archivo.
df_tweets_l = df_tweets

In [13]:
# Se observa que hay ausencias, sin embargo no son una cantidad significativa:
df_tweets_l.isnull().sum()

Date              0
text             16
Sentiment    891144
dtype: int64

In [14]:
# Eliminación registros con ausencias
df_tweets_l = df_tweets_l.dropna()
# Eliminación de registros duplicados.
df_tweets_l = df_tweets_l.drop_duplicates()

In [15]:
df_tweets_l = df_tweets_l[df_tweets_l.Sentiment != "Neutral"]

In [16]:
# Cantidad de datos y número de variables
df_tweets_l.shape

(15802921, 3)

In [17]:
df_tweets_l["Sentiment"].value_counts()

Negative    8013795
Positive    7789126
Name: Sentiment, dtype: int64

In [18]:
df_tweets_l["Date"]=pd.to_datetime(df_tweets_l["Date"], format='%Y-%m-%d', errors='coerce')

In [19]:
df_tweets_l["Date"].describe()

/tmp/ipykernel_27428/440195729.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_tweets_l["Date"].describe()


count                15802921
unique                   1892
top       2019-07-26 00:00:00
freq                   174149
first     2014-09-18 00:00:00
last      2019-11-23 00:00:00
Name: Date, dtype: object

## Precios

In [20]:
# Es recomendable que todos los pasos de limpieza y preparación se realicen sobre otro archivo.
df_precios_l = df_precios

In [21]:
# Se observa que hay ausencias, sin embargo no son una cantidad significativa:
df_precios_l.isnull().sum()

SNo          0
Name         0
Symbol       0
Date         0
High         0
Low          0
Open         0
Close        0
Volume       0
Marketcap    0
dtype: int64

In [22]:
df_precios_l["Date"]=pd.to_datetime(df_precios_l["Date"], format='%Y-%m-%d', errors='coerce').dt.date
df_precios_l["Date"]=pd.to_datetime(df_precios_l["Date"], format='%Y-%m-%d', errors='coerce')

In [23]:
df_precios_l["Date"].describe()

/tmp/ipykernel_27428/73369371.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_precios_l["Date"].describe()


count                    2991
unique                   2991
top       2013-04-29 00:00:00
freq                        1
first     2013-04-29 00:00:00
last      2021-07-06 00:00:00
Name: Date, dtype: object

In [24]:
df_precios_l["Variacion"]= df_precios_l["Close"]-df_precios_l["Open"]

In [25]:
df_precios_l.drop(["SNo", "Name", "Symbol", "High", "Low", "Open", "Close", "Volume", "Marketcap"], axis=1,inplace=True)

In [26]:
df_precios_l.dtypes

Date         datetime64[ns]
Variacion           float64
dtype: object

## Union de datasets

In [49]:
df_datos=df_tweets_l.merge(df_precios_l, left_on='Date', right_on='Date', how="left")

In [50]:
df_datos.head()

,Date,text,Sentiment,Variacion
0,2019-05-27,È appena uscito un nuovo video! LES CRYPTOMONN...,Positive,131.705973
1,2019-05-27,Cardano: Digitize Currencies; EOS https://t.co...,Positive,131.705973
2,2019-05-27,Another Test tweet that wasn't caught in the s...,Positive,131.705973
3,2019-05-27,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,Positive,131.705973
4,2019-05-27,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,Positive,131.705973


In [51]:
df_datos.describe()

,Variacion
count,1.580292e+07
mean,2.291801e+00
std,4.522804e+02
min,-2.345600e+03
25%,-1.763449e+02
50%,9.329834e-01
75%,1.896414e+02
max,3.633600e+03


In [52]:
percentile_positive=df_datos["Variacion"].quantile(0.66)
percentile_negative=df_datos["Variacion"].quantile(0.33)

In [53]:
df_datos.loc[df_datos["Variacion"]>percentile_positive, "Influencia"]= 1
df_datos.loc[df_datos["Variacion"]<percentile_negative, "Influencia"]= -1
df_datos.loc[(df_datos["Variacion"]>=percentile_negative) & (df_datos["Variacion"]<=percentile_positive), "Influencia"]= 0

In [54]:
df_datos["Influencia"].value_counts()

 1.0    5312443
 0.0    5284289
-1.0    5206189
Name: Influencia, dtype: int64

In [55]:
df_datos.drop( ["Date","Sentiment", "Variacion"],axis=1,inplace=True)

In [56]:
df_datos.head()

,text,Influencia
0,È appena uscito un nuovo video! LES CRYPTOMONN...,1.0
1,Cardano: Digitize Currencies; EOS https://t.co...,1.0
2,Another Test tweet that wasn't caught in the s...,1.0
3,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,1.0
4,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,1.0


In [61]:
df_datos.to_csv("./datos.csv", index=False)

# Modelo

In [62]:
df_modelo=pd.read_csv('./datos.csv', sep=',', encoding = 'utf-8', header=0)

In [63]:
df_modelo.shape

(15802921, 2)

In [64]:
df_modelo.head()

,text,Influencia
0,È appena uscito un nuovo video! LES CRYPTOMONN...,1.0
1,Cardano: Digitize Currencies; EOS https://t.co...,1.0
2,Another Test tweet that wasn't caught in the s...,1.0
3,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,1.0
4,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,1.0
